<a href="https://colab.research.google.com/github/sakurasakura1996/Pytorch-start-learning/blob/master/Dive_into_DL_Pytorch_4_2_%E5%8F%82%E6%95%B0%E5%88%9D%E5%A7%8B%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
from torch.nn import init

net = nn.Sequential(
    nn.Linear(4,3),
    nn.ReLU(),
    nn.Linear(3, 1)   # 就这样定义 pytorch已经进行默认初始化了
)
print(net)
X = torch.rand(2, 4)
Y = net(X).sum()   # 记得一定要在这里加上sum() 也就是返回的结果一定要为标量
# 不然后面反向传播时 Y.backward()就会报错了baby
print(Y)
print(Y.sum())

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)
tensor(0.0336, grad_fn=<SumBackward0>)
tensor(0.0336, grad_fn=<SumBackward0>)


In [0]:
# 访问模型参数
# 回忆Sequential类和Module类的继承关系，对于Sequential实例中含模型参数的层，我们可以通过
# Module类的parameters() or  named_parameters()方法来访问到所有的参数，以迭代器的形式放回。
# 后者除了返回参数Tensor外还会返回其名字。下面访问多层敢直接net的所有参数
print(type(net))
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

# 可见返回的名字自动加上了层数的索引作为前缀。

<class 'torch.nn.modules.container.Sequential'>
<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [0]:
# 我们再来访问net中单层的参数，对于使用Sequential 类构造的神经网络，我们可以通过方括号【】
# 来访问网络的任一层，索引0表示隐藏层为Sequential实例最先添加的层
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))
# parameters是param的类型，其实就是tensor的子类，和Tensor不同的是如果一个Tensor是
# Parameter类的话，那么它会自动被添加到模型的参数列表里，来看下麦呢这个例子

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [0]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20,20)
    
    def forward(self,x):
        pass

n = MyModel()
for name, param in n.named_parameters():
    print(name,param.size(),type(param))
    
# 这个例子就是说明 如果一个tensor是Paramter类的实例的话，它会自动加到 参数列表中去

weight1 torch.Size([20, 20]) <class 'torch.nn.parameter.Parameter'>


In [0]:
# 因为Parameter是Tensor，即tensor拥有的属性它都有，比如可以根据data来访问参数数值
# 用grad 来访问参数梯度
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)   # 反向传播前梯度为None
Y.backward()
print(weight_0.grad)

tensor([[-0.1987, -0.1055,  0.1257, -0.0772],
        [ 0.2914,  0.4856,  0.4213,  0.3935],
        [ 0.4958,  0.1483,  0.4524, -0.4767]])
None
tensor([[0.1094, 0.2646, 0.2896, 0.2278],
        [0.2249, 0.5438, 0.5952, 0.4682],
        [0.0751, 0.1816, 0.1988, 0.1564]])


In [0]:
# 初始化模型参数
# 之前提到pytorch中 nn.Module的模块参数都采取了较为合理的初始化策略。不同类型的layer采用的哪一种初始化方法去看源码
# 但我们经常需要用其他方法来初始化权重。pytorch中提供的init模块提供了多种预设的初始化方法。看下面例子
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[-0.0069, -0.0031, -0.0157, -0.0024],
        [-0.0140, -0.0005,  0.0058, -0.0035],
        [ 0.0204,  0.0002, -0.0059,  0.0009]])
2.weight tensor([[-0.0023,  0.0020, -0.0125]])


In [0]:
# 下面使用常数来初始化权重参数
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


In [0]:
# 如果只想对某个特定参数进行初始化，我们可以调用Parameter类的initialize函数，它与
# Block类提供的initialize函数使用方法一致，下例中我们对隐藏层的权重使用 Xavier随机初始化方法
# 4.2.3 自定义初始化方法
# 有时候初始化方法我们需要的 init模块中并没有提供，所以我们需要实现一个初始化方法，从而能够像使用其他初始化方法使用它，
# 在此之前我们先看看pytorch 怎么实现这些初始化方法的，例如 torch.nn.init.normal_
def normal_(tensor, mean=0, std=1):
    with torch.no_grad():
        return tensor.normal_(mean,std)
# 可以看到这就是一个in-place改变tensor值的函数，而且此过程不记录梯度。类似我们就可以实现自己定义的初始化方法。
# 下面例子中，我们令权重 有一半概率初始化为0，另一半概率初始化为[-10,-5]和[5,10]两个区间里均匀分布的随机数
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10,10)
        tensor *= (tensor.abs() >= 5).float()
        
for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)


0.weight tensor([[ 8.0981, -5.9484,  6.0992,  6.4422],
        [-8.1373, -9.6199,  0.0000, -0.0000],
        [ 0.0000, -0.0000,  9.6596,  0.0000]])
2.weight tensor([[-8.4161,  9.9235, -9.2910]])


In [0]:
# 此外，我们还可以通过改变这些参数的data来改写模型参数值同时不会影响梯度
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


In [0]:
# 4.2.4 共享模型参数
# 有些情况下，我们希望多个层之间共享模型参数。4.1.3提到了 Module类的forward函数里多次调用同一个层，此外如果我们传入
# Sequential的模块是同一个Module实例的话参数也是共享的
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear)
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)
# Sequential网络中有两个linear层，都是同一个实例，所以他们的参数是共享的，输出参数列表时也只有一个
# 在内存中，这两个线性层其实也是一个对象
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])
True
True


In [0]:
# 因为模型参数里面包含了梯度，所以在反向传播计算时，这些共享的参数的梯度是累加的
x = torch.ones(1,1)
y = net(x).sum()
print(x, y)
y.backward()
print(net[0].weight.grad)


tensor([[1.]]) tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
